In [1]:
import sqlite3

In [2]:
connection=sqlite3.connect("mydb.db")

In [3]:
connection

In [4]:
table_create_query = """
create table if not exists employee (
employee_id integer primary key autoincrement,
first_name text not null,
last_name text not null,
email text not null unique,
phone text not null unique,
salary real not null,
hire_date text not null
)
"""

In [5]:
table_create_query2 = """
create table if not exists customers (
customer_id integer primary key autoincrement,
first_name text not null,
last_name text not null,
email text not null unique,
phone text not null unique
)
"""

In [6]:
table_create_query3 = """
create table if not exists orders (
order_id integer primary key autoincrement,
customer_id integer not null,
order_date text not null,
amount real not null,
foreign key (customer_id) references customers(customer_id)
)
"""

In [7]:
cursor=connection.cursor()

In [8]:
cursor.execute(table_create_query)
cursor.execute(table_create_query2)
cursor.execute(table_create_query3)

In [9]:
insert_employee_query = """
insert into employee (first_name, last_name, email, phone, salary, hire_date) values (?, ?, ?, ?, ?, ?)
"""

insert_customer_query = """
insert into customers (first_name, last_name, email, phone) values (?, ?, ?, ?)
"""
insert_order_query = """
insert into orders (order_id, customer_id, order_date, amount) values (?, ?, ?, ?)
"""

In [10]:
employee_data = [
    ('adarsh', 'cv', 'adarsh.cv@example.com', '123-456-7890', 60000, '2020-01-15'),
    ('mithun', 'm', 'mithun.m@example.com', '987-654-3210', 75000, '2019-03-22'),
    ('darshan', 'naik', 'darshan.n@example.com', '555-123-4567', 80000, '2021-07-30'),
    ('kumar', 'gowda', 'kumar.g@example.com', '444-987-6543', 50000, '2022-11-05')
]

customer_data = [
    ('JOHN', 'DOE', 'john.doe@example.com', '123-456-7890'),
    ('JANE', 'SMITH', 'jane.smith@example.com', '987-654-3210'),
    ('ALICE', 'JOHNSON', 'alice.johnson@example.com', '555-123-4567'),
    ('BOB', 'BROWN', 'bob.brown@example.com', '444-987-6543')
]
order_data = [
    (1, 1, '2023-01-01', 100.50),
    (2, 2, '2023-02-15', 200.75),
    (3, 3, '2023-03-10', 150.00),
    (4, 4, '2023-04-20', 300.25)
]

In [11]:
cursor.executemany(insert_employee_query, employee_data)
cursor.executemany(insert_customer_query, customer_data)
cursor.executemany(insert_order_query, order_data)

In [12]:
connection.commit()

In [26]:
cursor.execute("select * from orders;")
for row in cursor.fetchall():
    print(row)

(1, 1, '2023-01-01', 100.5)
(2, 2, '2023-02-15', 200.75)
(3, 3, '2023-03-10', 150.0)
(4, 4, '2023-04-20', 300.25)


In [28]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///mydb.db")

In [30]:
db

In [31]:
db.dialect

'sqlite'

In [32]:
db.get_usable_table_names()

['customers', 'employee', 'orders']

In [39]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-70b-8192")

In [40]:
llm.invoke("How are YOU?")

AIMessage(content="I'm just a language model, I don't have feelings or emotions like humans do. I exist solely to process and respond to text-based input, so I don't have good or bad days or feelings like happiness or sadness. I'm simply here to provide information and assist with tasks to the best of my abilities!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 14, 'total_tokens': 79, 'completion_time': 0.185714286, 'prompt_time': 0.000173467, 'queue_time': 0.053779593, 'total_time': 0.185887753}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--3b364162-c7a3-4a41-a96d-651a8b8c86df-0', usage_metadata={'input_tokens': 14, 'output_tokens': 65, 'total_tokens': 79})

In [41]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [43]:
toolkit=SQLDatabaseToolkit(db=db, llm=llm)

In [44]:
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001275345E720>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001275345E720>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001275345E720>),
 QuerySQLCheckerTool(description='Use this tool to 

In [46]:
for tool in toolkit.get_tools():
    print(tool.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


In [47]:
list_tables_tools = next((tool for tool in toolkit.get_tools() if tool.name == "sql_db_list_tables"), None)

In [48]:
list_tables_tools

ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001275345E720>)

In [49]:
get_schema_tool = next((tool for tool in toolkit.get_tools() if tool.name == "sql_db_schema"), None)

In [50]:
get_schema_tool

InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001275345E720>)

In [54]:
list_tables_tools.invoke({})

'customers, employee, orders'

In [66]:
print(get_schema_tool.invoke("orders"))


CREATE TABLE orders (
	order_id INTEGER, 
	customer_id INTEGER NOT NULL, 
	order_date TEXT NOT NULL, 
	amount REAL NOT NULL, 
	PRIMARY KEY (order_id), 
	FOREIGN KEY(customer_id) REFERENCES customers (customer_id)
)

/*
3 rows from orders table:
order_id	customer_id	order_date	amount
1	1	2023-01-01	100.5
2	2	2023-02-15	200.75
3	3	2023-03-10	150.0
*/


In [75]:
from langchain_core.tools import tool

@tool
def db_query_tool(query: str) -> str:
    """Execute a SQL query on the database and return the result.
    If the query fails, return an error message.
    In case of error message , user will rewrite the query and try again."""
    result = db.run_no_throw(query)
    if not result:
        return "Error executing query. Please check the query syntax and try again."
    return result

In [77]:
print(db_query_tool.invoke("SELECT * FROM orders WHERE amount > 200"))
print(db_query_tool.invoke("SELECT * FROM employee WHERE salary > 60000"))

[(2, 2, '2023-02-15', 200.75), (4, 4, '2023-04-20', 300.25)]
[(2, 'mithun', 'm', 'mithun.m@example.com', '987-654-3210', 75000.0, '2019-03-22'), (3, 'darshan', 'naik', 'darshan.n@example.com', '555-123-4567', 80000.0, '2021-07-30')]
